In [ ]:
from google.colab import drive
import os
import pandas as pd
import re

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Path file metadata dan folder CSV serta output
base_folder = "/content/drive/My Drive/AYOK BIKIN DATASET"
csv_folder = os.path.join(base_folder, "[01] TABLE")
metadata_file = os.path.join(base_folder, "[00] METADATA/Metadata Tabel.xlsx")
output_folder = os.path.join(base_folder, "[03] DESCRIPTION")
os.makedirs(output_folder, exist_ok=True)  # Pastikan folder output ada

# Debugging untuk memeriksa path
print("Base folder:", base_folder)
print("CSV folder:", csv_folder)
print("Metadata file:", metadata_file)
print("Output folder:", output_folder)

Mounted at /content/drive
Base folder: /content/drive/My Drive/AYOK BIKIN DATASET
CSV folder: /content/drive/My Drive/AYOK BIKIN DATASET/[01] TABLE
Metadata file: /content/drive/My Drive/AYOK BIKIN DATASET/[00] METADATA/Metadata Tabel.xlsx
Output folder: /content/drive/My Drive/AYOK BIKIN DATASET/[03] DESCRIPTION


In [ ]:
# Fungsi untuk menentukan kata keterangan
def tentukan_kata_keterangan(judul_tabel):
    aturan = {
        "Jumlah": "sebanyak",
        "Luas": "sebesar",
        "Produksi": "sebanyak",
        "Daya": "sebesar",
        "Nilai": "sebesar",
        "Pertumbuhan": "sebesar",
        "Kepadatan": "sebesar",
        "Rasio": "sebesar",
        "Banyaknya": "sebanyak"
    }
    kata_pertama = judul_tabel.split()[0]
    return aturan.get(kata_pertama, "sebanyak")  # Default ke "sebanyak"


# Fungsi untuk mengekstrak satuan dari akhir judul tabel
def ekstrak_satuan(judul_tabel):
    # Temukan semua tanda kurung dalam judul
    matches = re.findall(r'\(([^()]+)\)', judul_tabel)

    if matches:
        satuan = matches[-1].strip()  # Ambil satuan terakhir yang paling dalam

        # Pastikan format satuan sesuai aturan
        if satuan in ["VA", "KWh"]:
            return satuan  # Tetap gunakan format aslinya
        elif satuan.lower().startswith("rp"):  # Menangani Rp. atau variasinya
            return "Rp"
        else:
            return satuan.lower()  # Ubah satuan lain ke huruf kecil

    return None  # Tidak ada satuan jika tidak ditemukan tanda kurung


# Fungsi untuk membuat judul tabel singkat
def buat_judul_tabel_singkat(judul_tabel):
    judul_tabel_singkat = re.sub(r'(?i)\s*Menurut Kecamatan.*', '', judul_tabel)
    judul_tabel_singkat = re.sub(r'(?i)\s*di Kota Malang.*', '', judul_tabel_singkat)
    return judul_tabel_singkat.strip()

In [ ]:
# Fungsi untuk mengekstrak satuan dari akhir judul tabel
def ekstrak_satuan(judul_tabel):
    # Temukan semua tanda kurung dalam judul
    matches = re.findall(r'\(([^()]+)\)', judul_tabel)

    if matches:
        satuan = matches[-1].strip()  # Ambil satuan terakhir yang paling dalam

        # Pastikan satuan hanya digunakan jika berada setelah "di Kota Malang" atau "Menurut Kecamatan"
        if "di Kota Malang" in judul_tabel:
            bagian_setelah_kota = judul_tabel.split("di Kota Malang")[-1]
            if matches[-1] not in bagian_setelah_kota:
                return None  # Abaikan jika tanda kurung tidak ada di akhir setelah "di Kota Malang"
        elif "Menurut Kecamatan" in judul_tabel:
            bagian_setelah_kecamatan = judul_tabel.split("Menurut Kecamatan")[-1]
            if matches[-1] not in bagian_setelah_kecamatan:
                return None  # Abaikan jika tanda kurung tidak ada di akhir setelah "Menurut Kecamatan"

        # Pastikan format satuan sesuai aturan
        if satuan in ["VA", "KWh"]:
            return satuan  # Tetap gunakan format aslinya
        elif satuan.lower().startswith("rp"):
            return "Rp"
        else:
            return satuan.lower()  # Ubah satuan lain ke huruf kecil

    return None  # Tidak ada satuan jika tidak ditemukan tanda kurung atau posisinya salah

In [ ]:
# 4. Membaca file metadata dari Excel
try:
    metadata = pd.read_excel(metadata_file)
    print("Metadata loaded successfully:")
    print(metadata.head())
except FileNotFoundError:
    print(f"Metadata file not found: {metadata_file}")
    raise

Metadata loaded successfully:
  Table ID                                        Judul Tabel  Tahun  \
0    T0001  Jumlah Kantor Pos Pembantu Menurut Kecamatan d...   2016   
1    T0002  Jumlah Kantor Pos Pembantu Menurut Kecamatan d...   2017   
2    T0003  Jumlah Kantor Pos Pembantu Menurut Kecamatan d...   2018   
3    T0004  Jumlah Kantor Pos Pembantu Menurut Kecamatan d...   2019   
4    T0005  Jumlah Kantor Pos Pembantu Menurut Kecamatan d...   2020   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  


In [ ]:
# 5. Iterasi berdasarkan metadata
for _, meta_row in metadata.iterrows():
    table_id = meta_row['Table ID']
    if not (table_id.startswith("T") and 1 <= int(table_id[1:]) <= 1200):
        continue

    judul_tabel = meta_row['Judul Tabel']
    tahun = meta_row['Tahun']
    kata_keterangan = tentukan_kata_keterangan(judul_tabel)
    satuan = ekstrak_satuan(judul_tabel)
    judul_tabel_singkat = buat_judul_tabel_singkat(judul_tabel)

    csv_file = os.path.join(csv_folder, f"{table_id}.csv")
    if os.path.exists(csv_file):
        data = pd.read_csv(csv_file, encoding='ISO-8859-1')
        kolom_kecamatan = data.columns[0]
        kolom_variabel = data.columns[1]

        deskripsi = f"Pada tahun {tahun}, {judul_tabel_singkat} "
        deskripsi_per_kecamatan = []

        for idx, row in data.iterrows():
            kecamatan = row[kolom_kecamatan]
            nilai = row[kolom_variabel]

            if isinstance(nilai, str) and nilai.strip() == "-":
                print(f"Kecamatan dengan nilai '-' dilewati pada baris {idx + 1}.")
                continue

           # Pastikan nilai adalah float sebelum konversi
            try:
                nilai_float = float(nilai)
                if nilai_float.is_integer():
                    nilai_str = f"{int(nilai_float):,}".replace(',', '.')  # Hanya pemisah ribuan
                else:
                    nilai_str = f"{nilai_float:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.')  # Pemisah ribuan & desimal koma
            except ValueError:
                nilai_str = str(nilai).strip()

            if kecamatan.upper() != "KOTA MALANG":
                if idx == len(data) - 2:
                    deskripsi_per_kecamatan.append(
                        f"dan di Kecamatan {kecamatan} {kata_keterangan} " +
                        (f"{satuan}{nilai_str}" if satuan == "Rp" else f"{nilai_str} {satuan}" if satuan else f"{nilai_str}")
                    )
                else:
                    deskripsi_per_kecamatan.append(
                        f"di Kecamatan {kecamatan} {kata_keterangan} " +
                        (f"{satuan}{nilai_str}" if satuan == "Rp" else f"{nilai_str} {satuan}" if satuan else f"{nilai_str}")
                    )
            else:
                total_kota = nilai_str

        deskripsi += ", ".join(deskripsi_per_kecamatan)
        deskripsi += f". Oleh karena itu, {judul_tabel_singkat} di Kota Malang"
        deskripsi += f" {kata_keterangan} " + (f"{satuan}{total_kota}" if satuan == "Rp" else f"{total_kota} {satuan}" if satuan else f"{total_kota}") + "."

        output_file = os.path.join(output_folder, f"{table_id}.txt")
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(deskripsi)
        print(f"Deskripsi untuk {table_id} berhasil disimpan.")

Deskripsi untuk T0001 berhasil disimpan.
Deskripsi untuk T0002 berhasil disimpan.
Deskripsi untuk T0003 berhasil disimpan.
Deskripsi untuk T0004 berhasil disimpan.
Deskripsi untuk T0005 berhasil disimpan.
Deskripsi untuk T0006 berhasil disimpan.
Deskripsi untuk T0007 berhasil disimpan.
Deskripsi untuk T0008 berhasil disimpan.
Deskripsi untuk T0009 berhasil disimpan.
Deskripsi untuk T0010 berhasil disimpan.
Deskripsi untuk T0011 berhasil disimpan.
Deskripsi untuk T0012 berhasil disimpan.
Deskripsi untuk T0013 berhasil disimpan.
Deskripsi untuk T0014 berhasil disimpan.
Deskripsi untuk T0015 berhasil disimpan.
Deskripsi untuk T0016 berhasil disimpan.
Deskripsi untuk T0017 berhasil disimpan.
Deskripsi untuk T0018 berhasil disimpan.
Deskripsi untuk T0019 berhasil disimpan.
Deskripsi untuk T0020 berhasil disimpan.
Deskripsi untuk T0021 berhasil disimpan.
Deskripsi untuk T0022 berhasil disimpan.
Deskripsi untuk T0023 berhasil disimpan.
Deskripsi untuk T0024 berhasil disimpan.
Deskripsi untuk 